1

In [ ]:
class GeometricDropout(nn.Module):
    def _init_(self, dropout_start=0.1, dropout_end=0.5, total_epochs=50):
        super(GeometricDropout, self)._init_()
        self.dropout_start = dropout_start
        self.dropout_end = dropout_end
        self.total_epochs = total_epochs

    def forward(self, x, epoch, training):
        if not training:
            return x
        dropout_rate = self.get_dropout_rate(epoch)
        noise = torch.empty_like(x).exponential_(dropout_rate)
        return x * (1 - noise)

    def get_dropout_rate(self, epoch):
        return self.dropout_start + (self.dropout_end - self.dropout_start) * (epoch / self.total_epochs)

2

In [ ]:
class NetStandard(nn.Module):
    def _init_(self, dropout_rate=0.5):
        super(NetStandard, self)._init_()
        self.conv1 = nn.Conv2d(3, 96, 5, padding=2)
        self.conv2 = nn.Conv2d(96, 128, 5, padding=2)
        self.conv3 = nn.Conv2d(128, 256, 5, padding=2)
        self.pool = nn.MaxPool2d(3, 2)
        self.fc1 = nn.Linear(256 * 3 * 3, 2048)
        self.fc2 = nn.Linear(2048, 2048)
        self.fc3 = nn.Linear(2048, 10)
        self.dropout_rate = dropout_rate

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 256 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        x = self.fc3(x)
        return x

3

In [ ]:
class NetAdaptive(nn.Module):
    def _init_(self, dropout_start=0.1, dropout_end=0.5, total_epochs=50):
        super(NetAdaptive, self)._init_()
        self.conv1 = nn.Conv2d(3, 96, 5, padding=2)
        self.conv2 = nn.Conv2d(96, 128, 5, padding=2)
        self.conv3 = nn.Conv2d(128, 256, 5, padding=2)
        self.pool = nn.MaxPool2d(3, 2)
        self.fc1 = nn.Linear(256 * 3 * 3, 2048)
        self.fc2 = nn.Linear(2048, 2048)
        self.fc3 = nn.Linear(2048, 10)
        self.geometric_dropout = GeometricDropout(dropout_start, dropout_end, total_epochs)

    def forward(self, x, epoch):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 256 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.geometric_dropout(x, epoch, self.training)  # Geometric dropout
        x = F.relu(self.fc2(x))
        x = self.geometric_dropout(x, epoch, self.training)  # Geometric dropout
        x = self.fc3(x)
        return x


4

In [ ]:
def calculate_sparsity(tensor):
    return 100.0 * float(torch.sum(tensor == 0)) / tensor.numel()

def track_sparsity(net, dataloader, epoch, adaptive):
    sparsity = 0.0
    count = 0
    net.eval()
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            if adaptive:
                outputs = net(images, epoch)
            else:
                outputs = net(images)
            sparsity += calculate_sparsity(outputs)
            count += 1
    return sparsity / count

def train_and_record_errors(net, trainloader, testloader, criterion, optimizer, num_epochs=50, adaptive=False):
    net.to(device)
    train_errors = []
    test_errors = []
    train_sparsity = []
    test_sparsity = []

    for epoch in range(num_epochs):
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            if adaptive:
                outputs = net(inputs, epoch)
            else:
                outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Calculate train and test error and sparsity
        train_error = calculate_classification_error(net, trainloader, epoch, adaptive)
        test_error = calculate_classification_error(net, testloader, epoch, adaptive)
        train_errors.append(train_error)
        test_errors.append(test_error)

        train_sparsity.append(track_sparsity(net, trainloader, epoch, adaptive))
        test_sparsity.append(track_sparsity(net, testloader, epoch, adaptive))

        print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}, Train Error: {train_error}, Test Error: {test_error}')
        print(f'Epoch {epoch + 1}, Train Sparsity: {train_sparsity[-1]}%, Test Sparsity: {test_sparsity[-1]}%')

    return train_errors, test_errors, train_sparsity, test_sparsity


5

In [ ]:
def calculate_sparsity(tensor):
    return 100.0 * float(torch.sum(tensor == 0)) / tensor.numel()

def track_sparsity(net, dataloader, epoch, adaptive):
    sparsity = 0.0
    count = 0
    net.eval()
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            if adaptive:
                outputs = net(images, epoch)
            else:
                outputs = net(images)
            sparsity += calculate_sparsity(outputs)
            count += 1
    return sparsity / count

def train_and_record_errors(net, trainloader, testloader, criterion, optimizer, num_epochs=50, adaptive=False):
    net.to(device)
    train_errors = []
    test_errors = []
    train_sparsity = []
    test_sparsity = []

    for epoch in range(num_epochs):
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            if adaptive:
                outputs = net(inputs, epoch)
            else:
                outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Calculate train and test error and sparsity
        train_error = calculate_classification_error(net, trainloader, epoch, adaptive)
        test_error = calculate_classification_error(net, testloader, epoch, adaptive)
        train_errors.append(train_error)
        test_errors.append(test_error)

        train_sparsity.append(track_sparsity(net, trainloader, epoch, adaptive))
        test_sparsity.append(track_sparsity(net, testloader, epoch, adaptive))

        print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}, Train Error: {train_error}, Test Error: {test_error}')
        print(f'Epoch {epoch + 1}, Train Sparsity: {train_sparsity[-1]}%, Test Sparsity: {test_sparsity[-1]}%')

    return train_errors, test_errors, train_sparsity, test_sparsity


6

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define your data loaders (assuming they are named trainloader and testloader)

# Train the NetStandard model with standard dropout
net_standard = NetStandard(dropout_rate=0.5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer_standard = optim.SGD(net_standard.parameters(), lr=0.001, momentum=0.9)
train_errors_standard, test_errors_standard, train_sparsity_standard, test_sparsity_standard = train_and_record_errors(
    net_standard, trainloader, testloader, criterion, optimizer_standard, num_epochs=10, adaptive=False)

# Train the NetAdaptive model with Geometric dropout
net_adaptive = NetAdaptive(dropout_start=0.1, dropout_end=0.5, total_epochs=10).to(device)
optimizer_adaptive = optim.SGD(net_adaptive.parameters(), lr=0.001, momentum=0.9)
train_errors_adaptive, test_errors_adaptive, train_sparsity_adaptive, test_sparsity_adaptive = train_and_record_errors(
    net_adaptive, trainloader, testloader, criterion, optimizer_adaptive, num_epochs=10, adaptive=True)